📌 Extracción

In [ ]:
#extraemos la data en formato json desde el API. Verificamos su estructura basica y configuracion
import pandas as pd

url = 'https://raw.githubusercontent.com/ingridcristh/challenge2-data-science-LATAM/refs/heads/main/TelecomX_Data.json'

df = pd.read_json(url)

df.head()
df.info()
df.shape

In [ ]:
#se necesita desanidar todas las columnas, por lo que cada columna es enviada a un diccionario y luego plasmada en una nueva tabla total
df_total = pd.json_normalize(df.to_dict(orient='records'))

df_total.head()
df_total.info()
df_total.shape

In [ ]:
#Hacemos una comprobación de incoherencias en los datos

#valores nulos: No se ubican valores nulos
df_total.isnull().sum().sort_values(ascending=False)

#registros duplicados, Primero verificamos por lineas y luego por CustomerID: No se ubican valores duplicados.
df_total.duplicated().sum()
df_total['customerID'].duplicated().sum()

#errores de formato por celdas: Se verifica que account.Charges.Total esta como object, sin embargo deberia ser una columna float
df_total.dtypes

#verificacion de columnas categoricas. 
    #- Churn tiene datos en blanco a parte de "no" y "yes"
    #- phone.MultipleLines tiene el dato "No phone service" a parte de "no" y "yes"
    #- internet.OnlineSecurity tiene el dato "No internet service" a parte de "no" y "yes"
    #- internet.OnlineBackup tiene el dato "No internet service" a parte de "no" y "yes"
    #- internet.DeviceProtection tiene el dato "No internet service" a parte de "no" y "yes"
    #- internet.TechSupport tiene el dato "No internet service" a parte de "no" y "yes"
    #- internet.StreamingTV tiene el dato "No internet service" a parte de "no" y "yes"
    #- internet.StreamingMovies tiene el dato "No internet service" a parte de "no" y "yes"

columnas_categoricas = df_total.select_dtypes(include='object').columns

for col in columnas_categoricas:
    print(f"\n{col}")
    print(df_total[col].value_counts())

In [ ]:
#Manejo de inconsistencias

#convertimos account.Charges.Total en columna numerica, estandarizando los campos en blanco con NaN

df_total['account.Charges.Total'] = (
    pd.to_numeric(df_total['account.Charges.Total'], errors='coerce')
)

df_total['account.Charges.Total'].fillna(0, inplace=True)

#normalizacion de variables categoricas: 
    #- columna Churn

df_total['Churn'] = df_total['Churn'].replace(' ', 'No')

    #- columna phone.MultipleLines

df_total['phone.MultipleLines'] = (
    df_total['phone.MultipleLines'].replace('No phone service', 'No')
)

    #- columnas que contienen "No internet service" en lugar de "No"

internet_cols = [
    'internet.OnlineSecurity',
    'internet.OnlineBackup',
    'internet.DeviceProtection',
    'internet.TechSupport',
    'internet.StreamingTV',
    'internet.StreamingMovies'
]

df_total[internet_cols] = df_total[internet_cols].replace('No internet service', 'No'
)

#validameos correcciones

for col in columnas_categoricas:
    print(f"\n{col}")
    print(df_total[col].value_counts())

df_total.dtypes
